In [1]:
@everywhere begin
    using DiffEqMonteCarlo, StochasticDiffEq, DiffEqBase, DiffEqProblemLibrary, DiffEqNoiseProcess, Plots, LaTeXStrings, ParallelDataTransfer

  probs = Matrix{SDEProblem}(3,3)
  p1 = Vector{Any}(3)
  p2 = Vector{Any}(3)
  p3 = Vector{Any}(3)
  ## Problem 1
  prob = prob_sde_linear
  probs[1,1] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM1)))
  probs[1,2] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM2)))
  probs[1,3] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM3)))
  ## Problem 2
  prob = prob_sde_wave
  probs[2,1] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM1)))
  probs[2,2] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM2)))
  probs[2,3] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM3)))
  ## Problem 3
  prob = prob_sde_additive
  probs[3,1] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM1)))
  probs[3,2] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM2)))
  probs[3,3] = SDEProblem(prob.f,prob.g,prob.u0,prob.tspan,noise=WienerProcess(0.0,0.0,0.0,rswm=RSWM(adaptivealg=:RSwM3)))
end

fullMeans = Vector{Array}(3)
fullMedians = Vector{Array}(3)
fullElapsed = Vector{Array}(3)
fullTols = Vector{Array}(3)

titleFontSize = 24
guideFontSize = 18
legendFontSize= 14
tickFontSize  = 16
Ns = [17 23 17]

1×3 Array{Int64,2}:
 17  23  17

Timings are only valid if no workers die. Workers die if you run out of memory.

In [2]:
for k in 1:size(probs,1)
  println("Problem $k")
  ## Setup
  N = Ns[k]

  msims = Vector{Any}(N)
  elapsed = Array{Float64}(N,3)
  medians = Array{Float64}(N,3)
  means   = Array{Float64}(N,3)
  tols    = Array{Float64}(N,3)

  #Compile
  prob = probs[k,1]
  ParallelDataTransfer.sendto(workers(), prob=prob)
  monte_prob = MonteCarloProblem(prob)
  solve(monte_prob,SRIW1(),dt=1/2^(4),adaptive=true,num_monte=1000,abstol=2.0^(-1),reltol=0)

  println("RSwM1")
  for i=1:N
    tols[i,1] = 2.0^(-i-1)
    msims[i] = calculate_monte_errors(solve(monte_prob,SRIW1(),dt=1/2^(4),
                                    adaptive=true,num_monte=1000,abstol=2.0^(-i-1),
                                    reltol=0,force_dtmin=true))
    elapsed[i,1] = msims[i].elapsedTime
    medians[i,1] = msims[i].error_medians[:final]
    means[i,1]   = msims[i].error_means[:final]
  end

  println("RSwM2")
  prob = probs[k,2]
  ParallelDataTransfer.sendto(workers(), prob=prob)
  monte_prob = MonteCarloProblem(prob)
  solve(monte_prob,SRIW1(),dt=1/2^(4),adaptive=true,num_monte=1000,abstol=2.0^(-1),reltol=0)

  for i=1:N
    tols[i,2] = 2.0^(-i-1)
    msims[i] = calculate_monte_errors(solve(monte_prob,SRIW1(),dt=1/2^(4),
                                    adaptive=true,num_monte=1000,abstol=2.0^(-i-1),
                                    reltol=0,force_dtmin=true))
    elapsed[i,2] = msims[i].elapsedTime
    medians[i,2] = msims[i].error_medians[:final]
    means[i,2]   = msims[i].error_means[:final]
  end

  println("RSwM3")
  prob = probs[k,3]
  ParallelDataTransfer.sendto(workers(), prob=prob)
  monte_prob = MonteCarloProblem(prob)
  solve(monte_prob,SRIW1(),dt=1/2^(4),adaptive=true,num_monte=1000,abstol=2.0^(-1),reltol=0)

  for i=1:N
    tols[i,3] = 2.0^(-i-1)
    msims[i] = calculate_monte_errors(solve(monte_prob,SRIW1(),dt=1/2^(4),
                                    adaptive=true,num_monte=1000,abstol=2.0^(-i-1),
                                    reltol=0,force_dtmin=true))
    elapsed[i,3] = msims[i].elapsedTime
    medians[i,3] = msims[i].error_medians[:final]
    means[i,3]   = msims[i].error_means[:final]
  end

  fullMeans[k] = means
  fullMedians[k] =medians
  fullElapsed[k] = elapsed
  fullTols[k] = tols
end


Problem 1
RSwM1
RSwM2
RSwM3
Problem 2
RSwM1
RSwM2
RSwM3
Problem 3
RSwM1
RSwM2
RSwM3


In [7]:
gr()
lw=3
leg=String["RSwM1","RSwM2","RSwM3"]'

for k in 1:size(probs,1)
  p1[k] = Plots.plot(fullTols[k],fullMeans[k],xscale=:log10,yscale=:log10,  xguide="Absolute Tolerance",yguide="Mean Final Error",title="Example $k"  ,linewidth=lw,grid=false,lab=leg,top_margin=50px,left_margin=100px,right_margin=50px,bottom_margin=50px,titlefont=font(titleFontSize),legendfont=font(legendFontSize),tickfont=font(tickFontSize),guidefont=font(guideFontSize))
  p2[k] = Plots.plot(fullTols[k],fullMedians[k],xscale=:log10,yscale=:log10,xguide="Absolute Tolerance",yguide="Median Final Error",title="Example $k",linewidth=lw,grid=false,lab=leg,top_margin=50px,left_margin=100px,right_margin=50px,bottom_margin=50px,titlefont=font(titleFontSize),legendfont=font(legendFontSize),tickfont=font(tickFontSize),guidefont=font(guideFontSize))
  p3[k] = Plots.plot(fullTols[k],fullElapsed[k],xscale=:log10,yscale=:log10,xguide="Absolute Tolerance",yguide="Elapsed Time",title="Example $k"      ,linewidth=lw,grid=false,lab=leg,top_margin=50px,left_margin=100px,right_margin=50px,bottom_margin=50px,titlefont=font(titleFontSize),legendfont=font(legendFontSize),tickfont=font(tickFontSize),guidefont=font(guideFontSize))
end

Plots.plot!(p1[1],margin=[20mm 0mm])
Plots.plot(p1[1],p1[2],p1[3],layout=(3,1),size=(1000,800),top_margin=50px,left_margin=100px,right_margin=50px)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Example 1 
 
 
 Absolute Tolerance 
 
 
 Mean Final Error 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 Example 2 
 
 
 Absolute Tolerance 
 
 
 Mean Final Error 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 Example 3 
 
 
 Absolute Tolerance 
 
 
 Mean Final Error 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3

In [4]:
#savefig("meanvstol.png")
#savefig("meanvstol.pdf")

In [8]:
plot(p3[1],p3[2],p3[3],layout=(3,1),size=(1000,800),top_margin=50px,left_margin=100px,right_margin=50px)
#savefig("timevstol.png")
#savefig("timevstol.pdf")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 Example 1 
 
 
 Absolute Tolerance 
 
 
 Elapsed Time 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 Example 2 
 
 
 Absolute Tolerance 
 
 
 Elapsed Time 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 Example 3 
 
 
 Absolute Tolerance 
 
 
 Elapsed Time 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3

In [9]:
plot(p1[1],p3[1],p1[2],p3[2],p1[3],p3[3],layout=(3,2),size=(1000,800),top_margin=50px,left_margin=100px,right_margin=50px)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Example 1 
 
 
 Absolute Tolerance 
 
 
 Mean Final Error 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 Example 1 
 
 
 Absolute Tolerance 
 
 
 Elapsed Time 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 Example 2 
 
 
 Absolute Tolerance 
 
 
 Mean Final Error 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 Example 2 
 
 
 Absolute Tolerance 
 
 
 Elapsed Time 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 Example 3 
 
 
 Absolute Tolerance 
 
 
 Mean Final Error 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 Example 3 
 
 
 Absolute Tolerance 
 
 
 Elapsed Time 
 
 
 
 
 
 
 
 
 RSwM1 
 
 
 
 RSwM2 
 
 
 
 RSwM3